In [2]:
import pandas as pd
from pathlib import Path
import requests
import json

from create_normalized_taxa import  create_taxa_df 


In [3]:
def log_df(df, nrows=5):
    print(df.shape)
    return df.head(nrows)

In [4]:
raw_data_paths = Path('../data').rglob('observations*.csv')
data_paths = [str(path) for path in raw_data_paths]
print(data_paths)

['../data/clarkstown-high-school-north/observations-200303.csv', '../data/cedar-creek-reserve/observations-199064.csv', '../data/los-angeles-bioblitz/observations-190446.csv', '../data/ciencia-ciudadana-peru-bats/observations-199065.csv', '../data/ciencia-ciudadana-peru-bees/observations-199066.csv']


# get unique quality_grade

In [7]:
for path in Path().rglob('../data/**/observations*.csv'):
    print(path)
    df = pd.read_csv(path, dtype=str)
    print(df['quality_grade'].unique())
    print(df['sound_url'].unique())


 

../data/clarkstown-high-school-north/observations-200303.csv
['research' 'casual' 'needs_id']
[nan]
../data/cedar-creek-reserve/observations-199064.csv
['casual' 'research' 'needs_id']
[nan 'https://static.inaturalist.org/sounds/193055.m4a?1619137769'
 'https://static.inaturalist.org/sounds/205842.m4a?1619980931']
../data/los-angeles-bioblitz/observations-190446.csv
['needs_id' 'research' 'casual']
[nan 'https://static.inaturalist.org/sounds/248114.m4a?1623137039'
 'https://static.inaturalist.org/sounds/249696.m4a?1623308146'
 'https://static.inaturalist.org/sounds/250018.wav?1623348240'
 'https://static.inaturalist.org/sounds/250071.wav?1623353309'
 'https://static.inaturalist.org/sounds/250074.wav?1623353572'
 'https://static.inaturalist.org/sounds/250076.wav?1623353796'
 'https://static.inaturalist.org/sounds/250791.m4a?1623441079'
 'https://static.inaturalist.org/sounds/251801.mp3?1623534008'
 'https://static.inaturalist.org/sounds/252473.m4a?1623597361'
 'https://static.inaturalis

---
# make taxa index species count match inaturalist

In [5]:
file = '../data/clarkstown-high-school-north/observations-200303.csv'
df = pd.read_csv(file, dtype=str)
log_df(df)

(1086, 42)


,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,created_at,updated_at,quality_grade,...,common_name,iconic_taxon_name,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_class_name,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_species_name
0,1551812,2015-05-27 08:30:00,2015-05-27,2015-05-27 05:30:00 -0700,Eastern Time (US & Canada),97128,lgertzer,2015-05-27 07:50:23 -0700,2019-05-12 16:17:45 -0700,research,...,Lettered Sphinx,Insecta,143452,Animalia,Arthropoda,Insecta,Lepidoptera,Sphingidae,Deidamia,Deidamia inscriptum
1,1556119,Thu May 28 2015 09:12:41 GMT-0400 (EDT),2015-05-28,2015-05-28 06:12:41 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:24 -0700,2016-08-29 10:44:52 -0700,casual,...,maples,Plantae,47727,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,NaN
2,1556120,Thu May 28 2015 09:14:17 GMT-0400 (EDT),2015-05-28,2015-05-28 06:14:17 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:43 -0700,2016-02-16 19:05:32 -0800,research,...,ribwort plantain,Plantae,53178,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
3,1556121,Thu May 28 2015 09:13:47 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:47 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2016-11-21 17:49:19 -0800,needs_id,...,Kentucky bluegrass,Plantae,60307,Plantae,Tracheophyta,Liliopsida,Poales,Poaceae,Poa,Poa pratensis
4,1556122,Thu May 28 2015 09:13:07 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:07 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2021-03-04 19:19:08 -0800,needs_id,...,dicots,Plantae,47124,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN


In [164]:
df['is_species'] = df['taxon_species_name'].notna()
adjust_is_species_for_higher_ranks(df)

In [165]:
inat_taxa = [
"Taraxacum officinale",
"Trifolium repens",
"Turdus migratorius",
"Rosa multiflora",
"Odocoileus virginianus",
"Sciurus carolinensis",
"Tamias striatus",
"Anas platyrhynchos",
"Branta canadensis",
"Trifolium pratense",
"Marmota monax",
"Glechoma hederacea",
"Plantago lanceolata",
"Artemisia vulgaris",
"Toxicodendron radicans",
"Camponotus pennsylvanicus",
"Harmonia axyridis",
"Parthenocissus quinquefolia",
"Dactylis glomerata",
"Ailanthus altissima",
"Nuphar",
"Poa pratensis",
"Rubus phoenicolasius",
"Cygnus olor",
"Agelaius phoeniceus",
"Sturnus vulgaris",
"Chelydra serpentina",
"Canis familiaris",
"Apis mellifera",
"Phytolacca americana",
"Armadillidium vulgare",
"Capsella bursa-pastoris",
"Rumex obtusifolius",
"Picea",
"Plantago rugelii",
"Formica fusca",
"Cyanocitta cristata",
"Passer domesticus",
"Homo sapiens",
"Theridiidae",
"Fagus grandifolia",
"Oxalis stricta",
"Liriodendron tulipifera",
"Hedera helix",
"Limenitis archippus",
"Verbascum thapsus",
"Acer palmatum",
"Marasmius oreades",
"Tilia cordata",
"Persicaria longiseta",
"Formica pallidefulva",
"Zenaida macroura",
"Ardea herodias",
"Corvus brachyrhynchos",
"Cardinalis cardinalis",
"Melospiza melodia",
"Quiscalus quiscula",
"Dumetella carolinensis",
"Vulpes vulpes",
"Sylvilagus floridanus",
"Hesperis matronalis",
"Iris pseudacorus",
"Ranunculus repens",
"Philoscia muscorum",
"Typha latifolia",
"Stellaria",
"Sisyrinchium angustifolium",
"Poa annua",
"Cyprinus carpio",
"Quercus alba",
"Chelidonium majus",
"Frangula alnus",
"Alliaria petiolata",
"Paeonia",
"Toxicodendron rydbergii",
"Plantago major",
"Rhododendron ponticum",
"Papilio glaucus",
"Limenitis arthemis",
"Xysticus",
"Lavandula",
"Viburnum acerifolium",
"Phragmites australis",
"Celastrus orbiculatus",
"Campaea perlata",
"Hosta",
"Periplaneta americana",
"Rhododendron maximum",
"Felis catus",
"Musca domestica",
"Entodon seductrix",
"Veronica officinalis",
"Platycladus orientalis",
"Ardea alba",
"Ranunculus fascicularis",
"Rhus typhina",
"Petunia × atkinsiana",
"Gallus gallus",
"Meleagris gallopavo",
"Aix sponsa",
"Spizella passerina",
"Piranga rubra",
"Molothrus ater",
"Hirundo rustica",
"Tachycineta bicolor",
"Sialia sialis",
"Hylocichla mustelina",
"Toxostoma rufum",
"Tyrannus tyrannus",
"Thamnophis",
"Chrysemys picta",
"Trachemys scripta",
"Terrapene carolina",
"Gopherus polyphemus",
"Procyon lotor",
"Canis latrans",
"Pardosa",
"Taxus",
"Salicaceae",
"Juniperus",
"Cypripedium acaule",
"Sherardia arvensis",
"Geranium maculatum",
"Acer negundo",
"Diplopoda",
"Acer rubrum",
"Castianeira",
"Scarabaeoidea",
"Thuja",
"Oniscus asellus",
"Vanessa cardui",
"Nepeta cataria",
"Oenothera",
"Prunus americana",
"Iris versicolor",
"Hemigrapsus sanguineus",
"Symplocarpus foetidus",
"Quercus rubra",
"Quercus velutina",
"Membracoidea",
"Kalmia",
"Platanus",
"Anthomyiidae",
"Linaria vulgaris",
"Cerastium",
"Chlorophyta",
"Erigeron",
"Moronidae",
"Veronica chamaedrys",
"Digitaria sanguinalis",
"Urtica dioica",
"Sphecius speciosus",
"Pinus resinosa",
"Acer saccharum",
"Equisetum hyemale",
"Prunella vulgaris",
"Achillea",
"Sarcophagidae",
"Cirsium vulgare",
"Hypochaeris radicata",
"Lactuca serriola",
"Vinca major",
"Cotoneaster pannosus",
"Ribes sanguineum",
"Trametes versicolor",
"Abies",
"Corylus avellana",
"Mentha",
"Schizophyllum commune",
"Acer platanoides",
"Quercus macrocarpa",
"Fraxinus",
"Celtis occidentalis",
"Rumex acetosa",
"Vinca minor",
"Lamium purpureum",
"Rosa canina",
"Leucanthemum",
"Robinia",
"Morus alba",
"Persicaria maculosa",
"Thelypteridaceae",
"Philaenus spumarius",
"Acer campestre",
"Ricinus communis",
"Veronica anagallis-aquatica",
"Callitriche",
"Chenopodium album",
"Trombidium",
"Anatrytone logan",
"Berberis thunbergii",
"Muhlenbergia rigens",
"Melissa officinalis",
"Cirsium arvense",
"Paspalum dilatatum",
"Nephrotoma",
"Smilax",
"Pluteus cervinus",
"Ipomoea pes-caprae",
"Chrysopilus thoracicus",
"Camponotus chromaiodes",
"Phalaris arundinacea",
"Dicentra eximia",
"Triticum aestivum",
"Lithobates catesbeianus",
"Geophilomorpha",
"Solidago canadensis",
"Carex vulpinoidea",
"Dysdera crocata",
"Aphaenogaster",
"Myrmica",
"Candelaria concolor",
"Pelargonium",
"Arctium lappa",
"Arenaria serpyllifolia",
"Bromus inermis",
"Buddleja davidii",
"Galium mollugo",
"Gypsophila elegans",
"Lepidium virginicum",
"Lonicera japonica",
"Phyllostachys aurea",
"Polygonum aviculare",
"Ranunculus bulbosus",
"Lumbricus terrestris",
"Potentilla canadensis",
"Platycryptus undatus",
"Halyomorpha halys",
"Rubus allegheniensis",
"Viola sororia",
"Hydrangea",
"Sambucus canadensis",
"Oxalis dillenii",
"Neoscona crucifera",
"Enallagma geminatum",
"Fundulus diaphanus",
"Ischnura posita",
"Rudbeckia laciniata",
"Euonymus alatus",
"Physcia",
"Porodaedalea pini",
"Calliphora vicina",
"Stropharia rugosoannulata",
"Meconema thalassinum",
"Quercus stellata",
"Vespula maculifrons",
"Sporobolus heterolepis",
"Condylostylus",
"Cupido comyntas",
"Plagiomnium",
"Helianthus tuberosus",
"Narcissus papyraceus",
"Ellychnia",
"Ranunculus hispidus",
"Stomolophus meleagris",
"Plutella",
"Securigera varia",
"Carex grayi",
"Bombus vagans",
"Asarum europaeum",
"Zelus luridus",
"Iresine",
"Menispermum",
"Coenosia",
"Chamaecyparis obtusa",
"Buxus sempervirens",
"Philonotis fontana",
"Deidamia inscriptum",
"Menidia menidia",
"Flavoparmelia baltimorensis",
"Sergiolus capulatus",
"Blatta orientalis",
"Sandalus niger",
"Alticini",
"Brassica carinata",
"Carex pedunculata",
"Clematis campaniflora",
"Dianthus chinensis",
"Hieracium murorum",
"Jungermannia exsertifolia",
"Miscanthus sinensis",
"Salvia nemorosa",
"Scolochloa festucacea",
"Viola × wittrockiana",
"Syrphinae",
"Cerrena unicolor",
"Rhododendron indicum",
"Equus caballus",
"Armadillidium nasatum",
"Celiptera frustulum",
"Podabrus tricostatus",
"Tabanus americanus",
"Potentilla indica",
"Avenella flexuosa",
"Rhagio mystaceus",
"Pluteus petasatus",
"Pseudeuophrys erratica",
"Myrmentoma",
"Festuca longifolia",
"Chrysoclista linneella",
"Coryphoideae",
"Iris × germanica",
"Pilosella caespitosa",
"Lysimachia arvensis",
"Dryobates pubescens",
"Lasius",
"Syringa komarowii",
"Schizomantodea",
"Candolleomyces candolleanus"
]

In [166]:
set(inat_taxa) - set(df[df['is_species']]['scientific_name'].unique())
# Iris × germanica
# Membracoidea
# Myrmentoma
# Scarabaeoidea
# Viola × wittrockiana

{'Chrysemys picta',
 'Gallus gallus',
 'Jungermannia exsertifolia',
 'Limenitis arthemis',
 'Membracoidea',
 'Myrmentoma',
 'Polygonum aviculare',
 'Rumex acetosa',
 'Scarabaeoidea',
 'Terrapene carolina'}

In [167]:
set(df[df['is_species']]['scientific_name'].unique()) - set(inat_taxa)


{'Anas platyrhynchos domesticus',
 'Chrysemys picta picta',
 'Formica',
 'Gallus gallus domesticus',
 'Jungermannia exsertifolia cordifolia',
 'Limenitis arthemis astyanax',
 'Polygonum aviculare aviculare',
 'Rumex acetosa acetosa',
 'Terrapene carolina carolina'}

---
# read projects.json

In [171]:
path = Path('../inat_data_explorer/src/lib/data/data.json')
projects_data = pd.read_json(path)
projects_data[0:1]

,username,card_title,card_summary,image,projects
0,mbarton,Los Angeles Bioblitz Challenge,Tempor sint irure incididunt sit voluptate lab...,https://picsum.photos/id/244,"[{'id': 1, 'title': 'Los Angeles Bioblitz Chal..."


In [172]:
files_project = {}

for projects in projects_data['projects']:
    for project in projects:
        for link in project['links']:
            if  'file' not in link:
                continue
            files_project[link['file']] = project['slug']

print(files_project)


{}


---
# fix time

Safari has invalid date error when using `new Date()` with the default iNat date format. Change to standard date format.

In [45]:
file = '../data/clarkstown-high-school-north/observations-200303.csv'
df = pd.read_csv(file )
log_df(df)

(1086, 42)


,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,created_at,updated_at,quality_grade,...,common_name,iconic_taxon_name,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_class_name,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_species_name
0,1551812,2015-05-27 08:30:00,2015-05-27,2015-05-27 05:30:00 -0700,Eastern Time (US & Canada),97128,lgertzer,2015-05-27 07:50:23 -0700,2019-05-12 16:17:45 -0700,research,...,Lettered Sphinx,Insecta,143452.0,Animalia,Arthropoda,Insecta,Lepidoptera,Sphingidae,Deidamia,Deidamia inscriptum
1,1556119,Thu May 28 2015 09:12:41 GMT-0400 (EDT),2015-05-28,2015-05-28 06:12:41 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:24 -0700,2016-08-29 10:44:52 -0700,casual,...,maples,Plantae,47727.0,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,NaN
2,1556120,Thu May 28 2015 09:14:17 GMT-0400 (EDT),2015-05-28,2015-05-28 06:14:17 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:43 -0700,2016-02-16 19:05:32 -0800,research,...,ribwort plantain,Plantae,53178.0,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
3,1556121,Thu May 28 2015 09:13:47 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:47 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2016-11-21 17:49:19 -0800,needs_id,...,Kentucky bluegrass,Plantae,60307.0,Plantae,Tracheophyta,Liliopsida,Poales,Poaceae,Poa,Poa pratensis
4,1556122,Thu May 28 2015 09:13:07 GMT-0400 (EDT),2015-05-28,2015-05-28 06:13:07 -0700,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2021-03-04 19:19:08 -0800,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN


In [33]:
df['id']

0        1551812
1        1556119
2        1556120
3        1556121
4        1556122
          ...   
1081    83431861
1082    83431862
1083    83431865
1084    83431868
1085    83683271
Name: id, Length: 1086, dtype: int64

In [38]:
df['time_observed_at'] = df['time_observed_at'].replace('([\d-]+) ([\d:]+) (-\d{4})', r'\1T\2Z', regex = True)
df['time_observed_at'] = df['time_observed_at'].astype(str)

In [39]:
df['time_observed_at']

0       2015-05-27T05:30:00Z
1       2015-05-28T06:12:41Z
2       2015-05-28T06:14:17Z
3       2015-05-28T06:13:47Z
4       2015-05-28T06:13:07Z
                ...         
1081    2021-06-04T13:46:00Z
1082    2021-06-04T13:46:00Z
1083    2021-06-04T13:46:00Z
1084    2021-06-04T13:46:00Z
1085    2021-06-04T07:12:48Z
Name: time_observed_at, Length: 1086, dtype: object

In [40]:
df

,id,observed_on_string,observed_on,time_observed_at,time_zone,user_id,user_login,created_at,updated_at,quality_grade,...,common_name,iconic_taxon_name,taxon_id,taxon_kingdom_name,taxon_phylum_name,taxon_class_name,taxon_order_name,taxon_family_name,taxon_genus_name,taxon_species_name
0,1551812,2015-05-27 08:30:00,2015-05-27,2015-05-27T05:30:00Z,Eastern Time (US & Canada),97128,lgertzer,2015-05-27 07:50:23 -0700,2019-05-12 16:17:45 -0700,research,...,Lettered Sphinx,Insecta,143452.0,Animalia,Arthropoda,Insecta,Lepidoptera,Sphingidae,Deidamia,Deidamia inscriptum
1,1556119,Thu May 28 2015 09:12:41 GMT-0400 (EDT),2015-05-28,2015-05-28T06:12:41Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:24 -0700,2016-08-29 10:44:52 -0700,casual,...,maples,Plantae,47727.0,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,NaN
2,1556120,Thu May 28 2015 09:14:17 GMT-0400 (EDT),2015-05-28,2015-05-28T06:14:17Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:43 -0700,2016-02-16 19:05:32 -0800,research,...,ribwort plantain,Plantae,53178.0,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
3,1556121,Thu May 28 2015 09:13:47 GMT-0400 (EDT),2015-05-28,2015-05-28T06:13:47Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2016-11-21 17:49:19 -0800,needs_id,...,Kentucky bluegrass,Plantae,60307.0,Plantae,Tracheophyta,Liliopsida,Poales,Poaceae,Poa,Poa pratensis
4,1556122,Thu May 28 2015 09:13:07 GMT-0400 (EDT),2015-05-28,2015-05-28T06:13:07Z,Eastern Time (US & Canada),105635,mcgovernm97,2015-05-28 06:14:25 -0700,2021-03-04 19:19:08 -0800,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,83431861,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:27 -0700,2021-06-17 13:54:27 -0700,needs_id,...,Red-flowering Currant,Plantae,53435.0,Plantae,Tracheophyta,Magnoliopsida,Saxifragales,Grossulariaceae,Ribes,Ribes sanguineum
1082,83431862,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:27 -0700,2021-10-13 19:21:37 -0700,research,...,ribwort plantain,Plantae,53178.0,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Plantaginaceae,Plantago,Plantago lanceolata
1083,83431865,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:28 -0700,2021-08-28 05:25:10 -0700,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN
1084,83431868,2021/06/04 4:46 PM EDT,2021-06-04,2021-06-04T13:46:00Z,Eastern Time (US & Canada),4586286,maxkahann,2021-06-17 13:54:28 -0700,2021-07-12 10:11:15 -0700,needs_id,...,dicots,Plantae,47124.0,Plantae,Tracheophyta,Magnoliopsida,NaN,NaN,NaN,NaN


In [42]:
df.to_json('temp/foo.json')

# qa LA indictor species

In [10]:
taxa_file = '../data/los-angeles-bioblitz/indicator_species.tsv'

indicator_df = pd.read_csv(taxa_file, sep='\t', dtype=str)
log_df(indicator_df)

(38, 8)


,image,image_credit,common_name,taxon_name,taxon_group,type,inat_link,taxon_id
0,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,California Quail,Callipepla californica,Birds,Park/Natural Area Species,https://www.inaturalist.org/taxa/1409-Callipep...,1409
1,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Greater Roadrunner,Geococcyx californianus,Birds,Park/Natural Area Species,https://www.inaturalist.org/taxa/1986-Geococcy...,1986
2,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Great Blue Heron,Ardea herodias,Birds,Stream/Riparian Species,https://www.inaturalist.org/taxa/4956-Ardea-he...,4956
3,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Red-tailed Hawk,Buteo jamaicensis,Birds,Neighborhood Species,https://www.inaturalist.org/taxa/5212-Buteo-ja...,5212
4,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Hooded Merganser,Lophodytes cucullatus,Birds,Stream/Riparian Species,https://www.inaturalist.org/taxa/7109-Lophodyt...,7109


In [13]:
observation_file = '../data/los-angeles-bioblitz/observations-190446.csv'
cols = ['taxon_id', 'common_name', 'scientific_name']
df = pd.read_csv(observation_file, usecols=cols, dtype=str)
df = df.dropna(subset=['taxon_id'])
df = df.drop_duplicates()

log_df(df)


(2775, 3)


,scientific_name,common_name,taxon_id
0,Nyctaginaceae,four o'clock family,50256
1,Metaltella simoni,South American Toothed Hacklemesh Weaver,224975
2,Canis latrans,Coyote,42051
3,Buteo jamaicensis,Red-tailed Hawk,5212
5,Zenaida macroura,Mourning Dove,3454


In [18]:
merge_df = indicator_df.merge(df, on='taxon_id', suffixes=('_la', None), how='left')
log_df(merge_df)


(38, 10)


,image,image_credit,common_name_la,taxon_name,taxon_group,type,inat_link,taxon_id,scientific_name,common_name
0,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,California Quail,Callipepla californica,Birds,Park/Natural Area Species,https://www.inaturalist.org/taxa/1409-Callipep...,1409,Callipepla californica,California Quail
1,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Greater Roadrunner,Geococcyx californianus,Birds,Park/Natural Area Species,https://www.inaturalist.org/taxa/1986-Geococcy...,1986,NaN,NaN
2,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Great Blue Heron,Ardea herodias,Birds,Stream/Riparian Species,https://www.inaturalist.org/taxa/4956-Ardea-he...,4956,Ardea herodias,Great Blue Heron
3,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Red-tailed Hawk,Buteo jamaicensis,Birds,Neighborhood Species,https://www.inaturalist.org/taxa/5212-Buteo-ja...,5212,Buteo jamaicensis,Red-tailed Hawk
4,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Hooded Merganser,Lophodytes cucullatus,Birds,Stream/Riparian Species,https://www.inaturalist.org/taxa/7109-Lophodyt...,7109,NaN,NaN


In [22]:
merge_df[(merge_df['taxon_name'] != merge_df['scientific_name']) & merge_df['scientific_name'].isna()]

 

,image,image_credit,common_name_la,taxon_name,taxon_group,type,inat_link,taxon_id,scientific_name,common_name
1,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Greater Roadrunner,Geococcyx californianus,Birds,Park/Natural Area Species,https://www.inaturalist.org/taxa/1986-Geococcy...,1986,NaN,NaN
4,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Hooded Merganser,Lophodytes cucullatus,Birds,Stream/Riparian Species,https://www.inaturalist.org/taxa/7109-Lophodyt...,7109,NaN,NaN
7,https://lapl.org/sites/default/files/styles/me...,Nurit Katz,Western Meadowlark,Sturnella neglecta,Birds,Park/Natural Area Species,https://www.inaturalist.org/taxa/9535-Sturnell...,9535,NaN,NaN
12,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,Black-bellied Slender Salamander,Batrachoseps nigriventris,Amphibians,Stream/Riparian Species,https://www.inaturalist.org/taxa/27474-Batrach...,27474,NaN,NaN
18,https://lapl.org/sites/default/files/styles/me...,Jeff Adams Stauffer,Dusky Footed Woodrat,Neotoma fuscipes,Mammals,Park/Natural Area Species,https://www.inaturalist.org/taxa/44749-Neotoma...,44749,NaN,NaN
25,https://lapl.org/sites/default/files/styles/me...,iNaturalist/Phytographer,Sara Orangetip,Anthocharis sara,Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/59124-Anthoch...,59124,NaN,NaN
30,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,Behr's Metalmark,Apodemia virgulti,Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/119068-Apodem...,119068,NaN,NaN
34,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,Bramble Green Hairstreak,Callophrys dumetorum,Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/346970-Callop...,346970,NaN,NaN
35,https://lapl.org/sites/default/files/styles/me...,iNaturalist/Patrick Alexander,Coachwhip Snake,Coluber flagellum,Reptiles,Park/Natural Area Species,https://www.inaturalist.org/taxa/547193-Mastic...,547193,NaN,NaN
37,https://lapl.org/sites/default/files/styles/me...,iNaturalist/Dario,Northern Harrier,Circus hudsonius,Birds,Park/Natural Area Species,https://www.inaturalist.org/taxa/558446-Circus...,558446,NaN,NaN


In [23]:
merge_df[(merge_df['taxon_name'] != merge_df['scientific_name']) & merge_df['scientific_name'].notna()]


,image,image_credit,common_name_la,taxon_name,taxon_group,type,inat_link,taxon_id,scientific_name,common_name
22,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,Bumblebees,Bombus sp.,Invertebrates,Neighborhood Species,https://www.inaturalist.org/taxa/52775-Bombus,52775,Bombus,Bumble Bees
27,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,Harvester Ants,Pogonomyrmex (Genus),Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/69114-Pogonom...,69114,Pogonomyrmex,Harvester Ants
29,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,North American Jerusalem Ccrickets,Ammopelmatus sp.,Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/86400-Ammopel...,86400,Ammopelmatus,North American Jerusalem Crickets
31,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,Baja California Tree Frog,Pseudacris hypochondria,Amphibians,Stream/Riparian Species,https://www.inaturalist.org/taxa/120719-Pseuda...,120719,Pseudacris hypochondriaca,Baja California Tree Frog
33,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,El Segundo Blue Butterfly,Euphilotes battoides ssp. allyni,Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/179676-Euphil...,179676,Euphilotes battoides allyni,El Segundo Blue


In [29]:
merge_df[(merge_df['common_name_la'] != merge_df['common_name']) & merge_df['scientific_name'].notna()]


,image,image_credit,common_name_la,taxon_name,taxon_group,type,inat_link,taxon_id,scientific_name,common_name
22,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,Bumblebees,Bombus sp.,Invertebrates,Neighborhood Species,https://www.inaturalist.org/taxa/52775-Bombus,52775,Bombus,Bumble Bees
29,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,North American Jerusalem Ccrickets,Ammopelmatus sp.,Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/86400-Ammopel...,86400,Ammopelmatus,North American Jerusalem Crickets
32,https://lapl.org/sites/default/files/styles/me...,iNaturalist/Andre Giraldi,California Kingsnake,Lampropeltis californiae,Reptiles,Park/Natural Area Species,https://www.inaturalist.org/taxa/146199-Lampro...,146199,Lampropeltis californiae,California King Snake
33,https://lapl.org/sites/default/files/styles/me...,Daniel S. Cooper,El Segundo Blue Butterfly,Euphilotes battoides ssp. allyni,Invertebrates,Park/Natural Area Species,https://www.inaturalist.org/taxa/179676-Euphil...,179676,Euphilotes battoides allyni,El Segundo Blue


# check if taxa and obervation counts are correct

In [42]:
file = '../app/src/lib/data/clarkstown-high-school-north/taxa.json'
taxa_df = pd.read_json(file, dtype=str)

In [43]:
file_path = '../data/clarkstown-high-school-north/observations-200303.csv'

obs_df = pd.read_csv(file_path, dtype=str)



In [55]:
#  species

id = '47602'
print(obs_df[obs_df['taxon_id']== id].shape)
print(obs_df[obs_df['taxon_species_name']== 'Taraxacum officinale'].shape)
print(obs_df[obs_df['scientific_name']== 'Taraxacum officinale'].shape)

taxa_df[taxa_df['taxon_id']== id]

(36, 42)
(36, 42)
(36, 42)


,id,taxon_id,common_name,scientific_name,user_login,image_url,rank,taxon_ids,common_names,scientific_names,is_species,observations_count,taxa_count
1,81601360,47602,common dandelion,Taraxacum officinale,carolinem3,https://static.inaturalist.org/photos/13385127...,species,47126|211194|47124|47605|47604|47603|47602,"plants|vascular plants|dicots|sunflowers, bell...",Plantae|Tracheophyta|Magnoliopsida|Asterales|A...,True,36,36


In [54]:
# higher rank taxa that has observations

id = '47603'
print(obs_df[obs_df['taxon_id']== id].shape)
print(obs_df[obs_df['taxon_genus_name']== 'Taraxacum'].shape)

# Taraxacum is genus and section
print(obs_df[obs_df['scientific_name']== 'Taraxacum'].shape) 

taxa_df[taxa_df['taxon_id']== id]


(14, 42)
(52, 42)
(16, 42)


,id,taxon_id,common_name,scientific_name,user_login,image_url,rank,taxon_ids,common_names,scientific_names,is_species,observations_count,taxa_count
10,81601360,47603,dandelions,Taraxacum,carolinem3,https://static.inaturalist.org/photos/13385127...,genus,47126|211194|47124|47605|47604|47603,"plants|vascular plants|dicots|sunflowers, bell...",Plantae|Tracheophyta|Magnoliopsida|Asterales|A...,False,14,52


In [53]:
# higher rank taxa without observations

id = '47148'
print(obs_df[obs_df['taxon_id']== id].shape)
print(obs_df[obs_df['taxon_family_name']== 'Rosaceae'].shape)
print(obs_df[obs_df['scientific_name']== 'Rosaceae'].shape)

taxa_df[taxa_df['taxon_id']== id]

(0, 42)
(56, 42)
(0, 42)


,id,taxon_id,common_name,scientific_name,user_login,image_url,rank,taxon_ids,common_names,scientific_names,is_species,observations_count,taxa_count
602,81641798,47148,rose family,Rosaceae,juliaiscience,https://inaturalist-open-data.s3.amazonaws.com...,family,47126|211194|47124|47132|47148,"plants|vascular plants|dicots|roses, elms, fig...",Plantae|Tracheophyta|Magnoliopsida|Rosales|Ros...,None,0,56


In [52]:
# non-major rank taxa

id = '1095093'
print(obs_df[obs_df['taxon_id']== id].shape)
print(obs_df[obs_df['scientific_name']== 'Formica pallidefulva'].shape)

taxa_df[taxa_df['taxon_id']== id]

(3, 42)
(3, 42)


,id,taxon_id,common_name,scientific_name,user_login,image_url,rank,taxon_ids,common_names,scientific_names,is_species,observations_count,taxa_count
76,81559931,1095093,pallidefulva-group Field Ants,Formica pallidefulva,zachb5,https://inaturalist-open-data.s3.amazonaws.com...,complex,1|47120|47158|47201|47336|47339||1095093,"Animals|Arthropods|Insects|Ants, Bees, Wasps, ...",Animalia|Arthropoda|Insecta|Hymenoptera|Formic...,True,3,3
